# Numerical exercise - Quantum Mechanics
**By Martin Johnsrud**

## Theory  

In this exercise we study a wavefunction with a gaussian

$$
    \Psi(x, t) = \exp \bigg(- \frac{(x - x_0)}{2 \sigma_x^2} \bigg) \exp \big(i (k_0 x - \omega t) \big)
$$

For a free particle, we have

$$
    \omega = \frac{\hbar k_0^2}{2m}, \quad E = \hbar \omega
$$

## Implemetation

To make numerical approximations of quantum mechanical problems, we make it a discreet problem. We look at $N_x$ points at the $x$-axis, in the interval $[0, L]$, distributed equally with a disdance $\Delta x = L \,/ (N_x - 1)$, and time steps are made in descreet jumps as well. The wave function for a given time, $\Psi(x, t_0)$ is represented as a vector, $\pmb{\Psi^{(0)}} = \big[ \Psi_{j}\big]_{j = 0}^{N_x - 1}$, where the indecies corresponds to the value of the wave function at $j\Delta_x$, i.e. $\Psi_j = \Psi(j\Delta x)$. The Schrödinger equation can then be modeled using a discrete aproximation of the double derivative with a matrix $D^2$, such that

$$
    \big(D^2 \pmb{\Psi} \big)_j = \frac{\Psi_{j + 1} - 2  \Psi_{j} + \Psi_{j - 1}}{\Delta x ^2} 
$$

The Hilbert operator, 

$$
     \bar H =  - \frac{\hbar ^2}{2 m} \frac{\partial^2}{\partial x^2} + V(x),
$$

then becomes a matrix $\pmb{H}$ with elements

$$
    H_{i, i} = \frac{1}{m(\Delta x)^2} + V_i, \quad H_{i, j \pm 1} = -\frac{1}{2m(\Delta x)^2} \\
    H_{0, 0} = H_{N_x, N_x} = 0,
$$
Where the first and last elements make the boundary condittion. Time eveloution of the wave function from time $t_k$ to $t_{k + 1}$ then becomes
$$
    \pmb{\Psi^{(k + 1)}} = \pmb{\Psi^{(k)}} - i \Delta t \pmb{H} \pmb{\Psi^{(k)}}
$$

### Notation

`Nx` - Number of points used in the discretization  
`x` - Array of length $N_x$ representing the x-axis

In [ ]:
import numpy as np
from numpy import exp, sqrt 
from numpy.linalg import norm
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.animation import FuncAnimation as FA
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

L = 20
k0 = 20
hbar = 1
m = 1

Nx = 1000
x = np.linspace(0, L, Nx)
dx = x[1] - x[0]
Sx = 1.5
x0 = 5

w = hbar * k0 / (2 * m)
dt = hbar / (hbar**2 / (2 * m) * 1 / dx**2) / 10
print("{:E}".format(dx))
print("{:E}".format(dt))

In [ ]:
H = np.zeros((Nx, Nx))
H[0, 0] = H[-1, -1] = 0
V = np.zeros(Nx)

for i in range(1, Nx - 1):
    H[i, i - 1] = H[i, i + 1] = - 1 / dx**2 + V[i]
    H[i, i] = + 2 / dx**2

In [ ]:
def timeStep(R, I):
    I = I - dt * H @ R
    R = R + dt * H @ I
    return (R, I)

def expexctation(x, Psi):
    return x @ abs(Psi)**2

def initial(x, t):
    return np.exp(-(x - x0)**2 / (2 * Sx**2)) * np.exp(1j * (k0 * x - w * t))

## Problem 1

In [ ]:
timeSteps = 5
Psi = np.zeros((timeSteps, 2, Nx))
R = initial(x, dt / 2).real
I = initial(x, 0).imag

C = norm(R + 1j* I)
R *= 1 / C
I *= 1 / C

fig, ax = plt.subplots(2, figsize = (20, 10))
ax[0].plot(x, R)
ax[0].plot(x, I)
ax[1].plot(x, sqrt(R**2 + I**2))
plt.show()
plt.close(fig)

## Problem 2

The group velocisty is given by

$$
    v_g = \frac{\partial \omega}{\partial k} \bigg\rvert_{k_0} = \frac{\hbar k_0}{m},
$$

so the time for the wave-packet to propagate a distance $l$, $T$, is given by

$$
    T = \frac{l}{v_g} = \frac{ml}{\hbar k_0}
$$

We propagate the wave from $x_0 = 5$ to $x = 15$, so that the time is given by $T = 10 m / \hbar k_0$, and the number of steps to take is

$$
    n = \frac{T}{\Delta t}
$$

In [ ]:
T = 10 * m / (hbar * k0)
print(T)
timeSteps = int(T / dt)
Psi = np.zeros((timeSteps, 2, Nx))
Psi[0, 0] = R
Psi[0, 1] = I
print(timeSteps)
for i in range(timeSteps - 1):
    Psi[i + 1] = timeStep(Psi[i, 0], Psi[i, 1])


In [ ]:
R = Psi[-1, 0]
I = Psi[-1, 1]

print(expexctation(x, R + 1j * I))

## Animation

In [ ]:
rcParams["animation.embed_limit"] =  "20"

def anim(i, fargs):
    Psi = fargs
    l[0].set_data(x, Psi[i, 0])
    l[0].set_3d_properties(Psi[i, 1])
    return l

stepsPerFrame = 50
Psi = Psi[::stepsPerFrame]

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection = "3d")
l = [ax.plot(x, Psi[0, 0], Psi[0, 1])[0] for i in range(1)]

a = FA(fig, anim, fargs = (Psi,), frames = len(Psi), interval = 50)

plt.close(fig)
HTML(a.to_jshtml())